# Questionnaire

Questionnaires rely on 4 classes:

* **Question**: A question to be addressed to the user. Receives user input, parses and stores it;
* **Response**: A response is attached to each question. It evaluates user input in execution time, formats and sends a response to the user.
* **Questionnaire**: Stores the graph-like structure of the questionnaire. Also, controls the flow of interaction with user;
* **Context**: Stores user collected data and state, for a REST communication.

### Questions

In [1]:
class Question:
    def __init__(self, name, **attr):
        self.name=name
        self.attr=attr
        
        self.prompt = attr.get("prompt")
        self.options = attr.get("options")
        self.options_text = attr.get("options_text")
        self.next = attr.get("next")
        self.user_input_type = attr.get("input_type")
        
        
    def getOptions(self):
        """
        Returns a dictionary mapping choices (int) to their respective texts.
        """
        if self.options is None: return None
        return { o:t for o,t in zip(self.options, self.options_text) }
    
    def saveUserInput(self, inpt):
        """
        Saves user response (input) to the question.
        input type: 'text','number','choice'
        """
        self.user_input = { 'text':str, 
                            'number':float, 
                            'choice':int }[self.user_input_type]( inpt )
        
    def isFinalQuestion(self):
        return True if self.next is None else False
    
    def attachResponse(self, response):
        self.response=response
        
    def respond(self):
        if not hasattr(self,'response'):
            return None
        
        return self.response.getResponse(self.user_input)

    
    def checkUserInput(self):
        return self.user_input
    
    def getNextQuestionName(self):
        """
        Checks user input to determine the next question in the graph.
        """
        if self.next is None:
            return None
        
        if len(self.next)>1:
            return self.next[ self.options.index(self.checkUserInput()) ]
        else:
            return self.next[0]
        
    def promptMessage(self):
        """
        Properly formats a message to be prompted to the user.
        """
        if self.user_input_type=='choice':
            return f"{self.prompt} {self.getOptions()}"
        else:
            return self.prompt

### Responses

Responses are associated to questions

In [2]:
class Response:
    def __init__(self, **attr):
        
        self.message=attr.get('message')
        self.condition=attr.get('condition')
        
        
    def getResponse(self, user_input):       
        if self.condition:
            return self.condition(user_input)
        else:
            return self.message

### Questionnaires

In [3]:
class Questionnaire:
    """
    Stores a graph of questions and runs it; keeps track of interactions using a context dictionary.
    """
    def __init__(self, questionnaire_dict=None):
        """
        Questionnaire dict to be used to build the instance
        """
        self.context = dict()
        
        if questionnaire_dict:
            self.questions = { qname: Question(qname,**qattrs) for qname, qattrs in questionnaire_dict.items() }
            
    
    def sendQuestion(self, context):
        questionId = context.data['current_question']
        q = self.questions.get(questionId)
        context.data['message']=q.promptMessage()
        context.data['user_input']=''
        context.data['input_type']=q.user_input_type
        
        input_options = q.options
        if input_options:
            context.data['input_options']=input_options
            
        input_options_text = q.options_text
        if input_options_text:
            context.data['input_options_text']=input_options_text

        return context
    
    def getUserInput(self, context):
        user_input = context.data['user_input']
        questionId = context.data['current_question']
        
        q = self.questions.get(questionId)
        q.saveUserInput(user_input)
        context.data['responses'][q.name]=q.checkUserInput()
        return context
    
    def sendResponse(self, context):
        questionId = context.data['current_question']
        q = self.questions.get(questionId)
        response = q.respond()
        context.data['message'] = response
        return context
    
    def getNextQuestion(self,context):
        currentQuestion = self.questions.get( context.data['current_question'] )
        nextQuestion = currentQuestion.getNextQuestionName()
        
            
        context.data['current_question'] = nextQuestion
        
        # clean non-relevant data from context
        context.data.pop('message',None)
        context.data.pop('user_input',None)
        context.data.pop('input_type',None)
        context.data.pop('input_options',None)
        context.data.pop('input_options_text',None)
        return context
    
    def run(self, context):
        finish=False
        
        while not finish:
            self.sendQuestion(context)
            user_input = input(context.data['message'])
            context.set_user_input(user_input)
            self.getUserInput(context)
            self.sendResponse(context)
            print(context.data['message'])
            self.getNextQuestion(context)
            
            next_q = context.data['current_question']
            if next_q is None:
                finish=True
        
        print("End of run")
        return context

### Context

In [4]:
class Context:
    def __init__(self, **data):
        self.data=data
        self.data['responses']={}
        
    def set_user_input(self,user_input):
        self.data['user_input']=user_input

---

## Building a questionnaire

One way to build and store a questionnaire is by using a JSON-like strucutre (dict in Python):

In [5]:
questionnaire = {
    'q1': {
        "prompt": "Oi! Me chamo Lando! Sou um assistente de terras. Qual seu nome?",
        "input_type": "text",
        "next":['q2']
          },
    'q2': {
        "prompt": "E sua idade?",
        "input_type": "number",
        "next":['q3']
    },
    'q3': {
        "prompt": "Primeiro preciso saber sobre a situação do seu solo. Ele está degradado?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": ["não","sim"],
        "next": ['q4']
    },
    'q4': {
        "prompt": "E pecuária? Tem pecuária na vizinhança?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": [ "não", "sim"],
        "next": ['q5']
    },
    'q5': {
        "prompt": "Tem risco de incêndio?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": [ "não", "sim"],
        "next": ['q6']
    },
    'q6': {
        "prompt": "Qual o potencial de regeneração natural?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": [ "baixo potencial", "alto potencial"],
        "next": ['q7','q8']
    },
    'q7':{
        "prompt": "Prefere plantar mudas ou semear?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": ["plantar mudas", "semear"],
        "next": ['q9', 'q10']
    },
    'q8':{
        "prompt": "Prefere manejar a regeneração natural ou não?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": ["não manejar", "manejar"]
    },
    'q9':{
        "prompt": "Que tipo de muda?",
        "input_type": "text"
    },
    'q10':{
        "prompt": "Que sementes?",
        "input_type": "text"
    }
}

Let's test it!

---

## Running the questionnaire: REST API messaging

In [6]:
# Build questionnaire and responses
q = Questionnaire(questionnaire)

# Response to q1
q.questions['q1'].attachResponse( Response( condition=lambda x: f"Prazer em conhecê-lo, {x}" ) )

# Response to q2
def cond(x):
    if x>=50:
        return "Nunca é tarde para começar!"
    elif x>=30:
        return "Boa idade!"
    elif x>=18:
        return "É ótimo começar cedo!"
    else:
        return "Você tem certeza que deveria estar usando este app?"
q.questions['q2'].attachResponse( Response( condition=cond ) )

# Response to q3
q.questions['q3'].attachResponse( Response( condition=lambda x: 'Certo... antes você precisará recuperar seu solo.' if x==1 else 'Ótimo!' ) )

# Response to q4
q.questions['q4'].attachResponse( Response( condition=lambda x: 'Entendo... é importante que vc cerque a área, ok?' if x==1 else 'Bom!') )

# Response to q5
q.questions['q5'].attachResponse( Response( condition=lambda x: 'Que tal construir um aceiro?' if x==1 else 'Bom!' ) )

In [7]:
# Build context
c = Context(current_question='q1')
c.data

{'current_question': 'q1', 'responses': {}}

### We send a question to user

In [8]:
q.sendQuestion(context=c)
c.data

{'current_question': 'q1',
 'responses': {},
 'message': 'Oi! Me chamo Lando! Sou um assistente de terras. Qual seu nome?',
 'user_input': '',
 'input_type': 'text'}

### User responds and we store input

In [9]:
c.set_user_input('Pedro')
c.data

{'current_question': 'q1',
 'responses': {},
 'message': 'Oi! Me chamo Lando! Sou um assistente de terras. Qual seu nome?',
 'user_input': 'Pedro',
 'input_type': 'text'}

data is sent to us...

In [10]:
q.getUserInput(context=c)
c.data

{'current_question': 'q1',
 'responses': {'q1': 'Pedro'},
 'message': 'Oi! Me chamo Lando! Sou um assistente de terras. Qual seu nome?',
 'user_input': 'Pedro',
 'input_type': 'text'}

### We send the response to user

In [11]:
q.sendResponse(context=c)
c.data

{'current_question': 'q1',
 'responses': {'q1': 'Pedro'},
 'message': 'Prazer em conhecê-lo, Pedro',
 'user_input': 'Pedro',
 'input_type': 'text'}

### User asks for the next question

In [12]:
q.getNextQuestion(context=c)
c.data

{'current_question': 'q2', 'responses': {'q1': 'Pedro'}}

---

### Running the entire questionnaire

In [13]:
c = Context(current_question='q1')
q.run(c)
c.data

Oi! Me chamo Lando! Sou um assistente de terras. Qual seu nome?Pedro
Prazer em conhecê-lo, Pedro
E sua idade?32
Boa idade!
Primeiro preciso saber sobre a situação do seu solo. Ele está degradado? {0: 'não', 1: 'sim'}1
Certo... antes você precisará recuperar seu solo.
E pecuária? Tem pecuária na vizinhança? {0: 'não', 1: 'sim'}0
Bom!
Tem risco de incêndio? {0: 'não', 1: 'sim'}1
Que tal construir um aceiro?
Qual o potencial de regeneração natural? {0: 'baixo potencial', 1: 'alto potencial'}0
None
Prefere plantar mudas ou semear? {0: 'plantar mudas', 1: 'semear'}1
None
Que sementes?não sei
None
End of run


{'current_question': None,
 'responses': {'q1': 'Pedro',
  'q2': 32.0,
  'q3': 1,
  'q4': 0,
  'q5': 1,
  'q6': 0,
  'q7': 1,
  'q10': 'não sei'}}